In [42]:
import os
import json
import shutil

import session_processing_helper_c5 as helper
import session_processing_helper_c5v2 as new_helper
import utils_c5 as utils

import pandas as pd
import statistics

enter cohort name and folder name

In [43]:
period = '20240531-20240806'
data_dir = '/Users/rebekahzhang/data/behavior_data'

# Backup clean data 

In [ ]:
def backup(source_dir, destination_dir):
    if not os.path.isdir(destination_dir):
        shutil.copytree(source_dir, destination_dir)
        print(f"{os.path.basename(source_dir)} folder backed up")
    else:
        print(f"{os.path.basename(destination_dir)} already exist")

In [ ]:
clean = os.path.join(data_dir, period, "clean")
clean_copy = os.path.join(data_dir, period, "clean_copy")
backup(clean, clean_copy)

# Sorting sessions to appopriate experiments and cohorts

In [3]:
data_folder = os.path.join(data_dir, period, "clean_copy")
print(data_folder)

/Users/rebekahzhang/data/behavior_data/20240531-20240806/clean_copy


load session log

In [11]:
sessions_all = utils.load_data(os.path.join(data_folder, 'sessions_all.csv'))
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

### First sort exp2 and exp3 seperately

In [7]:
def move_dir_to_new_folder(session_log, original_folder, target_folder):
    dir_list = session_log.dir.tolist()
    for dir in dir_list:
        session_dir = os.path.join(original_folder, dir)
        target_dir = os.path.join(target_folder, dir)

        if os.path.exists(session_dir) and not os.path.exists(target_dir):
            shutil.move(session_dir, target_dir)
        elif os.path.exists(target_dir):
            continue
        else:
            print(f"didn't work for: {dir}")

In [12]:
exp2_folder = os.path.join(data_dir, period, 'exp2')
exp3_folder = os.path.join(data_dir, period, 'exp3')

move everything for exp2 and save log there

In [13]:
# slice out logs for exp2
sessions_all_exp2 = sessions_all.loc[sessions_all.exp == 2]
sessions_training_exp2 = sessions_training.loc[sessions_training.exp == 2]
# specify exp2 folder
data_folder_exp2 = os.path.join(exp2_folder)
# move files from full_clean to exp folder
move_dir_to_new_folder(sessions_all_exp2, data_folder, data_folder_exp2)
# save sliced logs to exp_folde
utils.save_as_csv(df=sessions_all_exp2, folder=data_folder_exp2, filename='sessions_all_exp2.csv')
utils.save_as_csv(df=sessions_training_exp2, folder=data_folder_exp2, filename='sessions_training_exp2.csv')

move everything for exp3 and save log there

In [14]:
# slice out logs for exp3
sessions_all_exp3 = sessions_all.loc[sessions_all.exp == 3]
sessions_training_exp3 = sessions_training.loc[sessions_training.exp == 3]
# specify exp3 folder
data_folder_exp3 = os.path.join(exp3_folder)
# move files from full_clean to exp folder
move_dir_to_new_folder(sessions_all_exp3, data_folder, data_folder_exp3)
# save sliced logs to exp_folder
utils.save_as_csv(df=sessions_all_exp3, folder=data_folder_exp3, filename='sessions_all_exp3.csv')
utils.save_as_csv(df=sessions_training_exp3, folder=data_folder_exp3, filename='sessions_training_exp3.csv')

### Secondly sort between two cohorts of exp2 mice

In [49]:
cohort1 = ["RZ034", "RZ035", "RZ036", "RZ037", "RZ038", "RZ039"]
cohort2 = ["RZ047", "RZ048", "RZ049", "RZ050", "RZ051", "RZ052", "RZ053", "RZ054", "RZ055", "RZ056"]

In [50]:
sessions_all_exp2 = utils.load_data(os.path.join(exp2_folder, 'sessions_all_exp2.csv'))

In [51]:
sessions_all_exp2

,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group
590,2024-05-03,12-04-42,RZ036,2,regular,rig3,Rebekah,False,700,389,1.36,reward,2024-05-03_12-04-42_RZ036,s
353,2024-05-03,12-05-32,RZ034,2,regular,rig2,Rebekah,False,700,508,1.28,reward,2024-05-03_12-05-32_RZ034,s
439,2024-05-03,12-47-24,RZ037,2,regular,rig3,Rebekah,False,700,437,1.92,reward,2024-05-03_12-47-24_RZ037,l
512,2024-05-03,12-59-12,RZ038,2,regular,rig2,Rebekah,False,630,233,11.40,miss,2024-05-03_12-59-12_RZ038,l
93,2024-05-03,14-16-43,RZ039,2,regular,rig3,Rebekah,False,700,344,2.68,reward,2024-05-03_14-16-43_RZ039,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2024-08-06,13-58-50,RZ056,2,regular,rig2,Rebekah,False,700,509,1.15,reward,2024-08-06_13-58-50_RZ056,l
41,2024-08-06,14-00-34,RZ053,2,regular,rig3,Rebekah,False,245,89,7.51,miss,2024-08-06_14-00-34_RZ053,l
23,2024-08-06,14-41-06,RZ055,2,regular,rig3,Rebekah,False,190,69,4.92,miss,2024-08-06_14-41-06_RZ055,l
174,2024-08-06,15-08-13,RZ052,2,regular,rig3,Rebekah,False,300,183,2.16,miss,2024-08-06_15-08-13_RZ052,l


In [52]:
sessions_all_exp2_cohort1 = sessions_all_exp2[sessions_all_exp2.mouse.isin(cohort1)]
sessions_all_exp2_cohort2 = sessions_all_exp2[sessions_all_exp2.mouse.isin(cohort2)]

In [56]:
exp2_cohort1_folder = os.path.join(exp2_folder, 'cohort1')
move_dir_to_new_folder(sessions_all_exp2_cohort1, exp2_folder, exp2_cohort1_folder)

sessions_all = new_helper.generate_sessions_all(exp2_cohort1_folder)
sessions_training = new_helper.generate_sessions_training(sessions_all)

utils.save_as_csv(df=sessions_all, folder=data_folder, filename='sessions_all.csv')
utils.save_as_csv(df=sessions_training, folder=data_folder, filename='sessions_training.csv')

sessions_training

,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
0,197,2024-05-03,12-04-42,RZ036,2,regular,rig3,Rebekah,False,700,389,1.36,reward,2024-05-03_12-04-42_RZ036,s,0
1,108,2024-05-03,12-05-32,RZ034,2,regular,rig2,Rebekah,False,700,508,1.28,reward,2024-05-03_12-05-32_RZ034,s,0
2,143,2024-05-03,12-47-24,RZ037,2,regular,rig3,Rebekah,False,700,437,1.92,reward,2024-05-03_12-47-24_RZ037,l,0
3,165,2024-05-03,12-59-12,RZ038,2,regular,rig2,Rebekah,False,630,233,11.40,miss,2024-05-03_12-59-12_RZ038,l,0
4,30,2024-05-03,14-16-43,RZ039,2,regular,rig3,Rebekah,False,700,344,2.68,reward,2024-05-03_14-16-43_RZ039,l,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,0,2024-07-18,15-07-36,RZ038,2,regular,rig1,Rebekah,True,625,201,6.60,miss,2024-07-18_15-07-36_RZ038,l,43
211,210,2024-07-18,17-22-52,RZ039,2,regular,rig1,Rebekah,True,900,401,3.01,reward,2024-07-18_17-22-52_RZ039,l,39
212,133,2024-07-19,11-50-39,RZ037,2,regular,rig1,Rebekah,True,340,101,8.24,pygame,2024-07-19_11-50-39_RZ037,l,42
213,208,2024-07-19,12-38-14,RZ038,2,regular,rig1,Rebekah,True,505,176,9.30,miss,2024-07-19_12-38-14_RZ038,l,44


In [58]:
exp2_cohort2_folder = os.path.join(exp2_folder, 'cohort2')
move_dir_to_new_folder(sessions_all_exp2_cohort2, exp2_folder, exp2_cohort2_folder)

sessions_all = new_helper.generate_sessions_all(exp2_cohort2_folder)
sessions_training = new_helper.generate_sessions_training(sessions_all)

utils.save_as_csv(df=sessions_all, folder=data_folder, filename='sessions_all.csv')
utils.save_as_csv(df=sessions_training, folder=data_folder, filename='sessions_training.csv')

sessions_training

didn't work for: 2024-06-14_14-39-08_RZ055


,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
0,151,2024-06-17,10-38-55,RZ047,2,regular,rig2,Lianne,False,580.0,517.0,1.12,pygame,2024-06-17_10-38-55_RZ047,s,0
1,249,2024-06-17,10-51-54,RZ048,2,regular,rig3,Lianne,False,585.0,601.0,0.83,pygame,2024-06-17_10-51-54_RZ048,s,0
2,219,2024-06-17,11-30-07,RZ049,2,regular,rig2,Lianne,False,700.0,482.0,1.74,reward,2024-06-17_11-30-07_RZ049,s,0
3,320,2024-06-17,11-50-33,RZ050,2,regular,rig3,Lianne,False,580.0,409.0,1.48,pygame,2024-06-17_11-50-33_RZ050,s,0
4,10,2024-06-17,12-23-57,RZ051,2,regular,rig2,Lianne,False,700.0,545.0,1.16,reward,2024-06-17_12-23-57_RZ051,s,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,97,2024-08-06,13-58-50,RZ056,2,regular,rig2,Rebekah,False,700.0,509.0,1.15,reward,2024-08-06_13-58-50_RZ056,l,39
351,23,2024-08-06,14-00-34,RZ053,2,regular,rig3,Rebekah,False,245.0,89.0,7.51,miss,2024-08-06_14-00-34_RZ053,l,43
352,11,2024-08-06,14-41-06,RZ055,2,regular,rig3,Rebekah,False,190.0,69.0,4.92,miss,2024-08-06_14-41-06_RZ055,l,41
353,99,2024-08-06,15-08-13,RZ052,2,regular,rig3,Rebekah,False,300.0,183.0,2.16,miss,2024-08-06_15-08-13_RZ052,l,50


# Check for missing or multiple sessions for exp2 cohort 2

In [99]:
data_folder = os.path.join(data_dir, period, 'exp2', 'cohort2', 'clean')
print(data_folder)

/Users/rebekahzhang/data/behavior_data/20240531-20240806/exp2/cohort2/clean


In [100]:
sessions_all = new_helper.generate_sessions_all(data_folder)
sessions_all = sessions_all[sessions_all.mouse != "RZ048"]

sessions_training = new_helper.generate_sessions_training(sessions_all)

utils.save_as_csv(df=sessions_all, folder=data_folder, filename='sessions_all.csv')
utils.save_as_csv(df=sessions_training, folder=data_folder, filename='sessions_training.csv')

sessions_training

,index,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group,session
0,125,2024-06-17,10-38-55,RZ047,2,regular,rig2,Lianne,False,580.0,517.0,1.12,pygame,2024-06-17_10-38-55_RZ047,s,0
1,186,2024-06-17,11-30-07,RZ049,2,regular,rig2,Lianne,False,700.0,482.0,1.74,reward,2024-06-17_11-30-07_RZ049,s,0
2,275,2024-06-17,11-50-33,RZ050,2,regular,rig3,Lianne,False,580.0,409.0,1.48,pygame,2024-06-17_11-50-33_RZ050,s,0
3,9,2024-06-17,12-23-57,RZ051,2,regular,rig2,Lianne,False,700.0,545.0,1.16,reward,2024-06-17_12-23-57_RZ051,s,0
4,252,2024-06-17,12-36-01,RZ052,2,regular,rig3,Lianne,False,405.0,212.0,2.40,pygame,2024-06-17_12-36-01_RZ052,l,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,49,2024-08-06,11-36-58,RZ052,2,regular,rig2,Rebekah,False,370.0,203.0,2.39,miss,2024-08-06_11-36-58_RZ052,l,32
293,99,2024-08-06,11-38-34,RZ053,2,regular,rig3,Rebekah,False,650.0,322.0,2.81,miss,2024-08-06_11-38-34_RZ053,l,32
294,244,2024-08-06,12-34-42,RZ054,2,regular,rig2,Rebekah,False,700.0,337.0,3.15,reward,2024-08-06_12-34-42_RZ054,l,32
295,70,2024-08-06,13-00-03,RZ055,2,regular,rig3,Rebekah,False,395.0,179.0,2.88,miss,2024-08-06_13-00-03_RZ055,l,32


In [101]:
# generate a dict of number or rounds for each date
# mouse_list = utils.generate_mouse_list(sessions_all)
mouse_list = ['RZ047','RZ049','RZ050','RZ051','RZ052','RZ053','RZ054','RZ055','RZ056']
sessions_by_date = sessions_training.groupby('date')
rounds_dict = {}
for date, data in sessions_by_date:
    num_session_list = []
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        num_session_list.append(len(mouse_by_date))
    rounds_dict[date] = statistics.mode(num_session_list)

check for missing sessions, copy and paste from previous day to pad

In [102]:
no_missing_sessions = True
count = 0
for date, data in sessions_by_date:
    rounds = 1
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if len(mouse_by_date) < rounds:
            no_missing_sessions = False
            count += 1
            print(f"{date}: {mouse}")
if no_missing_sessions:
    print("no missing sessions!")
else:
    print(f"{count} sessions missing")

no missing sessions!


In [103]:
days_to_stitch = []
mice_to_stitch = []
for date, data in sessions_by_date:
    rounds = rounds_dict[date]
    for mouse in mouse_list:
        mouse_by_date = data.loc[data['mouse'] == mouse]
        if (len(mouse_by_date) > rounds) & (len(mouse_by_date) > 1):
            days_to_stitch.append(date)
            mice_to_stitch.append(mouse)
            print(f"on {date}, {mouse} has {len(mouse_by_date)} sessions")
if not days_to_stitch:
    print("no sessions to stitch!")

no sessions to stitch!


In [104]:
# run it if session stitching is needed, nothing would happen otherwise
if not days_to_stitch:
    print("no sessions to stitch!")
else:
    for d, m in zip(days_to_stitch, mice_to_stitch):
        day = sessions_by_date.get_group(d)
        sessions_to_stitch = day[day['mouse'] == m]

        session_1_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[0])
        session_2_dir = utils.generate_events_processed_path(data_folder, sessions_to_stitch.iloc[1])

        if os.path.exists(session_1_dir) and os.path.exists(session_2_dir):
            session_1 = pd.read_csv(session_1_dir)
            session_2 = pd.read_csv(session_2_dir)
            stitched_session = helper.stitch_sessions(session_1, session_2) 
            #should change to stitch events. stitch sessions should be deleted. to follow nomanclature, all session should be renamed to events!!

            stitched_session.to_csv(session_1_dir, index=False)
            shutil.rmtree(os.path.join(data_folder, sessions_to_stitch.iloc[1].dir))
            print(f"{d} {m} session 2 deleted")
        else:
            print("one of the sessions do not exist")

no sessions to stitch!


# regenerate sessions_all adding session time

In [127]:
data_folder

'/Users/rebekahzhang/data/behavior_data/20240531-20240806/exp2/cohort2/clean'

In [132]:
sessions_training = utils.load_data(os.path.join(data_folder, 'sessions_training.csv'))

In [133]:
sessions_all = utils.load_data(os.path.join(data_folder, 'sessions_all.csv'))

In [135]:
sessions_all

,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,avg_tw,ending_code,dir,group
33,2024-06-05,12-50-16,RZ047,2,shaping,rig2,Lianne,False,700.0,166.0,28.64,reward,2024-06-05_12-50-16_RZ047,s
23,2024-06-05,14-26-28,RZ049,2,shaping,rig2,Lianne,False,410.0,91.0,30.61,pygame,2024-06-05_14-26-28_RZ049,s
12,2024-06-05,15-19-48,RZ050,2,shaping,rig3,Lianne,False,365.0,79.0,29.19,pygame,2024-06-05_15-19-48_RZ050,s
246,2024-06-05,15-23-30,RZ051,2,shaping,rig2,Lianne,False,595.0,136.0,30.08,pygame,2024-06-05_15-23-30_RZ051,s
215,2024-06-06,12-20-27,RZ052,2,shaping,rig3,Lianne,False,415.0,96.0,30.56,pygame,2024-06-06_12-20-27_RZ052,l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,2024-08-06,11-36-58,RZ052,2,regular,rig2,Rebekah,False,370.0,203.0,2.39,miss,2024-08-06_11-36-58_RZ052,l
99,2024-08-06,11-38-34,RZ053,2,regular,rig3,Rebekah,False,650.0,322.0,2.81,miss,2024-08-06_11-38-34_RZ053,l
244,2024-08-06,12-34-42,RZ054,2,regular,rig2,Rebekah,False,700.0,337.0,3.15,reward,2024-08-06_12-34-42_RZ054,l
70,2024-08-06,13-00-03,RZ055,2,regular,rig3,Rebekah,False,395.0,179.0,2.88,miss,2024-08-06_13-00-03_RZ055,l


In [137]:
session_info = sessions_training.iloc[0]
events = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info))
session_basics = {'dir': session_info.dir} | helper.get_session_basics(events)


In [138]:
session_basics

{'dir': '2024-06-17_10-38-55_RZ047',
 'num_blocks': 1,
 'num_trials': 517,
 'rewards': 580.0,
 'session_time': 2868.09,
 'proper_end': False}

In [140]:
session_basics_list = []
for _, session_info in sessions_training.iterrows():
    try:
        events = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info))
        session_basics = {'dir': session_info.dir} | helper.get_session_basics(events)
        session_basics_list.append(session_basics)
    except:
        print(session_info.dir)
session_basics_df = pd.DataFrame(session_basics_list)
sessions_all = pd.merge(sessions_all, session_basics_df, on='dir')

In [126]:
sessions_all

,date,time,mouse,exp,training,rig,trainer,record,total_reward,total_trial,...,num_blocks_x,num_trials_x,rewards_x,session_time_x,proper_end_x,num_blocks_y,num_trials_y,rewards_y,session_time_y,proper_end_y
0,2024-06-17,10-38-55,RZ047,2,regular,rig2,Lianne,False,580.0,517.0,...,1,517,580.0,2868.09,False,1,517,580.0,2868.09,False
1,2024-06-17,11-30-07,RZ049,2,regular,rig2,Lianne,False,700.0,482.0,...,1,482,700.0,3009.11,False,1,482,700.0,3009.11,False
2,2024-06-17,11-50-33,RZ050,2,regular,rig3,Lianne,False,580.0,409.0,...,1,409,580.0,2545.49,False,1,409,580.0,2545.49,False
3,2024-06-17,12-23-57,RZ051,2,regular,rig2,Lianne,False,700.0,545.0,...,1,545,700.0,3242.83,False,1,545,700.0,3242.83,False
4,2024-06-17,12-36-01,RZ052,2,regular,rig3,Lianne,False,405.0,212.0,...,1,212,405.0,7017.28,False,1,212,405.0,7017.28,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,2024-08-06,11-36-58,RZ052,2,regular,rig2,Rebekah,False,370.0,203.0,...,2,386,670.0,18335.26,False,2,386,670.0,18335.26,False
293,2024-08-06,11-38-34,RZ053,2,regular,rig3,Rebekah,False,650.0,322.0,...,2,411,895.0,14801.49,False,2,411,895.0,14801.49,False
294,2024-08-06,12-34-42,RZ054,2,regular,rig2,Rebekah,False,700.0,337.0,...,1,337,700.0,4322.31,False,1,337,700.0,4322.31,False
295,2024-08-06,13-00-03,RZ055,2,regular,rig3,Rebekah,False,395.0,179.0,...,2,248,585.0,9206.18,False,2,248,585.0,9206.18,False
